### Naludaq Version
*Min Version*: `0.17.2`

In [ ]:
# Print Naludaq version
import naludaq
print(f"Naludaq version: {naludaq.__version__}")

### Prerequisites
+ Acquisition captured previously (can be from NaluScope)

In [ ]:
# Run this code for plotting
import matplotlib.pyplot as plt

from naluexamples.helpers.plotting import get_color_mapping, set_plot_style

def plot_event(event: dict, channels: list[int]=None, title: str=None):
    """Plot an event dict.

    Args:
        event (dict): the parsed event
        channels (list[int]): Channels to plot. Defaults to all channels.
        title (str): Optional title for the plot
    """
    channels = channels or range(len(event["data"]))
    title = title or f"Event #{event.get('pkg_num', '?')}"


    # Plot the event
    plt.figure(figsize=(8,8), constrained_layout=True)
    plt.xlabel("Time")
    plt.ylabel("ADC Counts")
    plt.title(title)
    cmap = get_color_mapping("ocean")
    set_plot_style(font_size=18, font_family="monospace")
    for i, channel in enumerate(channels):
        time = event["time"][channel]
        data = event["data"][channel]
        plt.plot(time, data, ".-", label=f"Channel {channel}", color=cmap(i / len(channels)))
    plt.legend()
    plt.show()

## Acquisition Format

Acquisitions captured using the Nalu Scientific hardware server use a different data format than legacy acquisitions in order to support higher readout rates and add some failsafe mechanisms

```
acquisition/
├── metadata.yml - Stores board parameters and register values at time of capture
├── {n}.bin - One or more "chunks" containing raw events stored back-to-back in chronological order. See below for more information
├── {n}.idx - Contains information about where in the `.bin` files each event is located. See below for more information
├── (pedestals_calibration) - Pedestals calibration data, stored as a Python pickle
├── (timing_calibration) - Timing calibration data, stored as a Python pickle
└── (adc2mv_calibration) - ADC/mV calibration data, stored as a Python pickle
```

### Bin File Format
Each `.bin` file is referred to as a "chunk." Each chunk is limited to 500 MB and begins with the following header:

```rust
// Struct is written according to native endianness
struct Header {
    // Format revision number
    version: u16,
    // Reserved for future use
    _reserved: u16,
    // Length of the metadata sector
    metadata_sector_length: u32,
}
```

The metadata sector follows the header, and contains a copy of the `metadata.yml` file.

### Idx File Format
Each `.idx` file is referred to as an "index file," and contains a long list of "entries" holding information about where each event is located in the chunk file with the corresponding name. Each entry has the following format:

```rust
// Struct is written according to native endianness
struct IndexEntry {
    // Offset in bytes of the event in the chunk.
    offset: u32,
    // Length of the event
    length: u32,
}
```

### Opening Acquisitions

This example applies to acquisitions captured using the server. For legacy acquisitions, see the other notebook.

In [ ]:
from naludaq.backend import DiskAcquisition


ACQ_PATH = r"<PATH TO ACQUISITION FOLDER>"

# DiskAcquisition is recommended to be used as a context manager for safety
with DiskAcquisition(ACQ_PATH) as acq:
    print("Board model:", acq.params["model"])
    print("Number of events:", len(acq))
    print("Pedestals stored:", acq.pedestals is not None)

Events can be accessed in either raw or parsed form. For most purposes, parsed events are desired. Acquisitions are not loaded into memory because they can be very large. As a consequence, this means they currently are loaded from disk each time. If this is a concern, it is recommended to cache the results.

In [ ]:
EVENT_INDEX = 0

# Read the event from disk
with DiskAcquisition(ACQ_PATH) as acq:
    # Accessing events with the subscript operator will parse the events automatically.
    # The subscript operator also supports slices.
    EVENT = acq[EVENT_INDEX]

# Missing data field means the event could not be parsed
if EVENT.get("data", None) is None:
    print("Event is corrupted!")
else:
    plot_event(EVENT)

## Pedestals Subtraction

For both acquisition formats, the events stored in the acquisitions are not pedestal-subtracted. To obtain pedestals-subtracted events, the process must be performed manually on the events.

In [ ]:
from naludaq.tools.pedestals.pedestals_correcter import PedestalsCorrecter


# Fetch an event, pedestals, and params from disk
ACQ_PATH = r"<PATH TO ACQUISITION FOLDER>"
EVENT_INDEX = 0
with DiskAcquisition(ACQ_PATH) as acq:
    event = acq[EVENT_INDEX]
    params = acq.params
    pedestals = acq.pedestals
    
if pedestals is None:
    print("Acquisition must have pedestals in order to correct data!")
else:
    # Apply correction (in place)
    corrector = PedestalsCorrecter(params, pedestals)
    corrected_event = corrector.run(event, correct_in_place=True)
    plot_event(corrected_event)